In [45]:
import pandas as pd 
import ast
from sklearn.preprocessing import MinMaxScaler
import os

os.chdir('C:/Users/shara/Projects/2025_Siam-ML-Hack')

In [52]:

def add_feature(row):
    base_path = "src/raw_data"
    
    file_name = row.iloc[0]
    recovery_range = ast.literal_eval(row.iloc[1])
    drop_range = ast.literal_eval(row.iloc[2])

    train_path = os.path.join(base_path, "train", file_name)
    test_path = os.path.join(base_path, "test", file_name)

    if os.path.exists(train_path):
        file_path = train_path
    else: 
        file_path = test_path

    df = pd.read_csv(file_path, names=["time", "pressure"], header=None, sep="\t")

    scaler = MinMaxScaler()
    df[['time', 'pressure']] = scaler.fit_transform(df[['time', 'pressure']])

    df['recovery_flag'] = 0
    df['drop_flag'] = 0

    for start, end in recovery_range:
        norm_start, norm_end = scaler.transform([[start, df['pressure'].min()]])[0][0], scaler.transform([[end, df['pressure'].min()]])[0][0]
        df.loc[(df['time'] >= norm_start) & (df['time'] <= norm_end), 'recovery_flag'] = 1

    for start, end in drop_range:
        norm_start, norm_end = scaler.transform([[start, df['pressure'].min()]])[0][0], scaler.transform([[end, df['pressure'].min()]])[0][0]
        df.loc[(df['time'] >= norm_start) & (df['time'] <= norm_end), 'drop_flag'] = 1

    print(df)
    return df


ground_truth = pd.read_csv('src/raw_data/ground_truth.csv')
row = ground_truth.iloc[0]
df_result = add_feature(row)

           time  pressure  recovery_flag  drop_flag
0      0.000000  0.297505              0          0
1      0.000003  0.297505              0          0
2      0.000055  0.297931              0          0
3      0.000058  0.297931              0          0
4      0.000125  0.297931              0          0
...         ...       ...            ...        ...
30997  0.999915  0.108552              0          0
30998  0.999953  0.108445              0          0
30999  0.999957  0.108445              0          0
31000  0.999986  0.108445              0          0
31001  1.000000  0.107379              0          0

[31002 rows x 4 columns]


c:\Users\shara\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\shara\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\shara\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\shara\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\shara\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fit